In [1]:
import pandas as pd
import numpy as np
import regex as re
import pandas as pd
import emoji
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from bltk.langtools import Tokenizer
from bltk.langtools import UgraStemmer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from bangla_stemmer.stemmer import stemmer
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_excel('data.xlsx')
#dataset = pd.read_excel('data.xlsx')
dataset = dataset.dropna()
dataset = dataset[0:5000]
#dataset = dataset[['comment','label']]
print(len(dataset))

5000


In [3]:
dataset

,label,comment
0,0,মাদ্রিদ হারার পর পুরা মনটাই ভাইঙ্গা গেছে :( কি...
1,0,আমি মানুষটা খুব একটা ভালো না। আসলে ভালো থেকে ক...
2,0,নোয়াখালীতে একজন মারা গেছে পিকেটারদের ইটের আঘাত...
3,0,﻿মন দিতে না পারলে মন ভেঙো না। সুখ দিতে না পারল...
4,0,আজাইরা কামে যে প্রচুর সময় নষ্ট করি তা আজকেও আর...
...,...,...
4995,0,এরা ব্লেয়ারাইট ডানপন্থী।আজকেই বৈষম্যমূলক মন্তব...
4996,0,ক্রিকেট ছেড়ে বিশ্বরাজনীতি নিয়ে পড়েছেন? নাকি টা...
4997,0,সব দেশ যদি এমন শুরু করে তাহলে আমরা যাবটা কথায়?
4998,0,মার্কিন নিষেধাজ্ঞার কারণে ডলারের বিপরীতে ইরানি...


In [4]:
bangla_stop_words =["অতএব", "অথচ", "অথবা", "অনুযায়ী", "অনেক", "অনেকে", "অনেকেই", "অন্তত", "অবধি", "অবশ্য", 
"আগামী", "আগে", "আগেই", "আছে", "আজ", "আদ্যভাগে", "আপনার", "আপনি", "আবার", "আমরা", 
"আমাকে", "আমাদের", "আমার", "আমি", "আর", "আরও", "ইত্যাদি", "ইহা", "উচিত", "উনি", "উপর", 
"উপরে", "এ", "এঁদের", "এঁরা", "এই", "এক", "একই", "একজন", "একটা", "একটি", "একবার", "একে", 
"এখন", "এখনও", "এখানে", "এখানেই", "এটা", "এটাই", "এটি", "এত", "এতটাই", "এতে", "এদের", "এবং", 
"এবার", "এমন", "এমনকি", "এর", "এরা", "এলো", "এস", "এসে", "ঐ", "ও", "ওঁদের", "ওঁর", "ওঁরা", 
"ওই", "ওকে", "ওখানে", "ওদের", "ওর", "ওরা", "কখনও", "কত", "কথা", "কবে", "কয়েক", "কয়েকটি", 
"করছে", "করছেন", "করতে", "করবে", "করবেন", "করলে", "করলেন", "করা", "করাই", "করায়", "করার", 
"করি", "করিতে", "করিয়া", "করিয়ে", "করে", "করেই", "করেছিলেন", "করেছে", "করেছেন", "করেন", "কাউকে", 
"কাছ", "কাছে", "কাজ", "কাজে", "কারও", "কারণ", "কি", "কিংবা", "কিছু", "কিছুই", "কিন্তু", "কী", "কে", 
"কেউ", "কেউই", "কেন", "কোন", "কোনও", "কোনো", "ক্ষেত্রে", "খুব", "গিয়ে", "গিয়েছে", "গুলি", "গেছে", 
"গেল", "গেলে", "গোটা", "চলে", "চান", "চায়", "চেয়ে", "ছাড়া", "ছাড়াও", "ছিল", "ছিলেন", "জন", "জনকে", 
"জনের", "জন্য", "জন্যে", "জানতে", "জানা", "জানানো", "জানায়", "জানিয়ে", "জানিয়েছে", "টি", "তখন", 
"তত", "তথা", "তবু", "তবে", "তা", "তাঁকে", "তাঁদের", "তাঁর", "তাঁরা", "তাঁহারা", "তাই", "তাও", "তাকে", "তাতে", 
"তাদের", "তার", "তারপর", "তারা", "তারই", "তাহলে", "তাহা", "তাহাতে", "তাহার", "তিনই", "তিনি", "তিনিও", 
"তুমি", "তুলে", "তেমন", "তো", "তোমার", "থাকবে", "থাকবেন", "থাকা", "থাকায়", "থাকে", "থাকেন", "থেকে", 
"থেকেই", "থেকেও", "দিকে", "দিতে", "দিয়ে", "দিয়েছে", "দিয়েছেন", "দিলেন", "দিয়ে", "দু", "দুটি", "দুটো", 
"দেওয়া", "দেওয়ার", "দেখতে", "দেখা", "দেখে", "দেন", "দেয়", "দেশের", "দ্বারা", "ধরা", "ধরে", 
"নাকি", "নাগাদ", "নানা", "নিজে", "নিজেই", "নিজেদের", "নিজের", "নিতে", "নিয়ে", "নিয়ে", "নেওয়া", 
"নেওয়ার", "পক্ষে", "পর", "পরে", "পরেই", "পরেও", "পর্যন্ত", "পাওয়া", "পারি", "পারে", "পারেন", "পেয়ে", 
"প্রতি", "প্রভৃতি", "প্রায়", "ফলে", "ফিরে", "ফের", "বছর", "বদলে", "বরং", "বলতে", "বলল", "বললেন", "বলা", "বলে", 
"বলেছেন", "বলেন", "বসে", "বহু", "বা", "বাদে", "বার", "বিনা", "বিভিন্ন", "বিশেষ", "বিষয়টি", "বেশ", "ব্যবহার", 
"ব্যাপারে", "ভাবে", "ভাবেই", "মত", "মতো", "মতোই", "মধ্যভাগে", "মধ্যে", "মধ্যেই", "মধ্যেও", "মনে", "মাত্র", "মাধ্যমে", 
"মানুষ", "মানুষের", "মোট", "মোটেই", "যখন", "যত", "যতটা", "যথেষ্ট", "যদি", "যদিও", "যা", "যাঁর", "যাঁরা", "যাওয়া", 
"যাওয়ার", "যাকে", "যাচ্ছে", "যাতে", "যাদের", "যান", "যাবে", "যায়", "যার", "যারা", "যায়", "যিনি", "যে", "যেখানে", 
"যেতে", "যেন", "যেমন", "রকম", "রয়েছে", "রাখা", "রেখে", "শুধু", "শুরু", "সঙ্গে", "সঙ্গেও", "সব", "সবার", "সমস্ত", 
"সম্প্রতি", "সময়", "সহ", "সহিত", "সাথে", "সুতরাং", "সে", "সেই", "সেখান", "সেখানে", "সেটা", "সেটাই", "সেটাও", "সেটি", 
"স্পষ্ট", "স্বয়ং", "হইতে", "হইবে", "হইয়া", "হওয়া", "হওয়ায়", "হওয়ার", "হচ্ছে", "হত", "হতে", "হতেই", "হন", "হবে", 
"হবেন", "হয়", "হয়তো", "হয়নি", "হয়ে", "হয়েই", "হয়েছিল", "হয়েছে", "হয়েছেন", "হল", "হলে", "হলেই", "হলেও", "হলো", 
"হিসাবে", "হিসেবে", "হৈলে", "হোক", "হয়", "হয়ে", "হয়েছে", "অর্থাৎ", "অন্য", "অনুযায়ী", "অর্ধভাগে", "এসো", "কয়েক", 
"কয়েকটি", "করিয়ে", "করিয়া", "করায়", "গিয়ে", "গিয়েছে", "ছাড়া", "ছাড়াও", "থাকায়", "দিয়েছে", "দিয়েছেন", "দেয়", "দেওয়া", 
"দেওয়ার", "নেওয়ার", "পাওয়া", "পেয়ে", "প্রায়", "বিষয়টি", "যাওয়া", "যাওয়ার", "রয়েছে", "স্বয়ং", "হৈতে", "হইয়া", "হয়েছিল", 
"হয়েছেন", "হয়নি", "হয়েই", "হয়তো", "হওয়া", "হওয়ার", "হওয়ায়", "জানায়", "জাানিয়ে", "জানিয়েছে", "চায়", "চেয়ে", "উত্তর", 
"এমনি", "কেমনে", "কোটি", "চার", "চালু", "চেষ্টা", "দিন", "দুই", "নতুন", "পাঁচ", "প্রথম", "প্রাথমিক", "বক্তব্য", "বন", "বেশি", 
"লক্ষ", "সাধারণ", "সামনে", "হাজার"]
names = ['বাংলাদেশের','ভারত','বাংলাদেশ','তাহসান','মিথিলা','সৃজিত','তাহসানের',
        'আয়রা','আব্দুল লতিফ','আরে','তাহসিন','শাকিব','জয়','রুবেল','দিঘীকে',
         'দিঘি','দীঘির','দিগি','জোলিল','জলিল','অনন্ত','শাকিব','খান','তাহসিরেশন',
         'অপুকে','অপুর','সাকিব','অপু','নাহিদ','আজহারি','জলিল','জিহাদি',
        'শ্রীজিৎ','অনন্ত ','','','','','','']
stop_words_for_own_dataset = ['আ‌মি','ভারতে','দে‌খছি','তোর','কারন',
                              'বুঝা','একদিন','মানে','গেলো','হুম','রাখে',
                              'কেমন','ই','টু','তো','কেমন','ওটা',
                              'মেয়েটাকে','মার','স্বামীর','নাম','তোমরা','দাদা',
                              'মহিলা','টা','চাই','টাও','র','টা','এদেরকে','র্থ',
                              'ভাই','ভাইয়া','হজুর','আপ্নে','এইটা','ু','তারে',
                              'তুই','তাসনিম','রে','ন','তুমার','তোমারে','কইরো',
                              'আপনাকে','নকে','ল','থা','র','লোক','ব','জয়ের',
                              'র','ভাইয়া','ভারত','ধর্ষকের','পোশাক','ধর্ষন',
                              'ধর্ষনের','ধর্ষন','ধর্ষক','','','']

In [5]:
corpus = []
#stemmer = UgraStemmer()
stmr = stemmer.BanglaStemmer()
for index,value in dataset.iterrows():
    text = str(value['comment'])
    text = re.sub(r'https?:\/\/\S+','',text) #removing hyperlinks
    text = re.sub("[\.\,\!\?\:\;\-\=।*/+-@#$%^&()]", " ", text)
    text = re.sub("[A-Za-z]"," ",text)
    text = re.sub("[0-9]"," ",text)
    text = re.sub("[০-৯]"," ",text)
    text = text.replace("'", "")
    text = text.replace('"', '')
    text = emoji.get_emoji_regexp().sub(r'', text)
      
    temp = []
    tokens = Tokenizer().word_tokenizer(text)
    for word in tokens:
        if word not in bangla_stop_words:
            if word not in stop_words_for_own_dataset:
                if word not in names:
                    temp.append(word)
    
    text = ' '.join(temp)
    
#     temp = []
#     tokens = Tokenizer().word_tokenizer(text)
#     for word in tokens:
#         stm = stmr.stem(word)
#         temp.append(stm)
#     text = ' '.join(temp)
    
#     tokens = Tokenizer().word_tokenizer(text)
#     stem = stemmer.stem(tokens)
#     text = ' '.join(stem)
    
    text = text.strip()
    corpus.append(text)

C:\Users\hp\AppData\Local\Temp/ipykernel_8960/1127518504.py:13: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  text = emoji.get_emoji_regexp().sub(r'', text)


In [6]:
corpus

['মাদ্রিদ হারার পুরা মনটাই ভাইঙ্গা ভালো লাগেনা তারিখ কোপেনহেগেন হোম ম্যাচ',
 'মানুষটা ভালো না আসলে ভালো সুবিধা না',
 'নোয়াখালীতে মারা পিকেটারদের ইটের আঘাতে',
 '\ufeffমন না পারলে মন ভেঙো না সুখ না পারলে কষ্ট দিওনা ভালোবাসতে না পারলে অভিনয় করিও না অভিনয় কারো জীবন নষ্ট করো না',
 'আজাইরা কামে প্রচুর নষ্ট আজকেও আরেকবার প্রমান হইলো কামের খালি ঘুম আসে সমস্যা',
 'লাইক ভিক্কা চাইছিল দি নাই গালা গালি করলো দরকার ভার্চুয়াল খ্যাতি সামান্য লাইক',
 'পিছনে ইচ্ছে ইস সত্যি পিছনে পারতাম_তাহলে ভুল গুলো শুধরে পারতাম',
 'বাংলা লায়ন বাল আজকের রোমান্টিক ওয়েদার মুখটা খারাপ করাইল',
 'এল স্বীকার্যগুলো মাথায় ঢুকছে না পুরাই মাথা নষ্ট ম্যান টাইপের জিনিস',
 'পোলাপাইন দেখলে অসহ্য লাগে',
 'মেয়েদের রাগ কম',
 'সবকিছুর নির্দিষ্ট সীমা ভালোবাসার মানুষকে মিস্ সময় কষ্ট সীমা না',
 'কাছের ভুলে সংক্ষিপ্ত পথ নেই প্রতিদিন পড়তে স্মৃতিগুলো কষ্ট',
 'মনটা খারাপ দিলাম দিল না',
 'প্রকৃতিতে বড়ই অসহায় প্রাণী নিয়তিকে মেনে',
 'বাহিরে বৃষ্টি হচ্ছে| বৃষ্টিতে ভিজতে ইচ্ছে করচ্ছে খুব|',
 'কষ্ট না পারো রেখ তোমাকেই কষ্ট পেতে',
 'খুবই অশান্ত

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(corpus).toarray()

In [8]:
y = dataset['label'].values

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 1)

In [10]:
o = 0
for i in y_test:
    if i == 1:
        o = o + 1
print(o)

416


In [11]:
## multinomial naive bayes for tf-idf

In [12]:
multinb_model = MultinomialNB()
multinb_model.fit(x_train,y_train)
y_pred_multinb = multinb_model.predict(x_test)
print(classification_report(y_test,y_pred_multinb))
print(confusion_matrix(y_test,y_pred_multinb))

              precision    recall  f1-score   support

           0       0.64      0.82      0.72       584
           1       0.59      0.36      0.45       416

    accuracy                           0.63      1000
   macro avg       0.62      0.59      0.59      1000
weighted avg       0.62      0.63      0.61      1000

[[480 104]
 [265 151]]


In [13]:
clf = svm.SVC(kernel='linear')
t = clf.fit(x_train, y_train)
y_pred_svm = t.predict(x_test) 
print('classification report of multinomial naiva bayes for tf-idf:\n\n\n',classification_report(y_test,y_pred_svm))

classification report of multinomial naiva bayes for tf-idf:


               precision    recall  f1-score   support

           0       0.64      0.78      0.70       584
           1       0.56      0.39      0.46       416

    accuracy                           0.62      1000
   macro avg       0.60      0.58      0.58      1000
weighted avg       0.61      0.62      0.60      1000



In [14]:
logreg_model = LogisticRegression()
logreg_model.fit(x_train,y_train)
y_pred_logreg = logreg_model.predict(x_test)
print('classification report of logistic regression for tf-idf:\n\n\n',classification_report(y_test,y_pred_logreg))
print(confusion_matrix(y_test,y_pred_logreg))

classification report of logistic regression for tf-idf:


               precision    recall  f1-score   support

           0       0.64      0.80      0.71       584
           1       0.57      0.36      0.44       416

    accuracy                           0.62      1000
   macro avg       0.60      0.58      0.58      1000
weighted avg       0.61      0.62      0.60      1000

[[469 115]
 [266 150]]


In [15]:
xgb_model = XGBClassifier()
xgb_model.fit(x_train, y_train)
y_pred_xgb = xgb_model.predict(x_test)
print('classification report of xgboost for tf-idf:\n\n\n',classification_report(y_test,y_pred_xgb))
print(confusion_matrix(y_test,y_pred_xgb))

classification report of xgboost for tf-idf:


               precision    recall  f1-score   support

           0       0.63      0.83      0.72       584
           1       0.58      0.33      0.42       416

    accuracy                           0.62      1000
   macro avg       0.60      0.58      0.57      1000
weighted avg       0.61      0.62      0.59      1000

[[483 101]
 [279 137]]
